In [97]:
import numpy as np
import matplotlib.pyplot as plt

l = 0.2 #length of container
#function to generate radial thermal resistance
def Rth_radial(r1,r2, k):
    Rth = np.log(r2/r1) / (2*np.pi*l*k)
    return Rth


In [98]:
#Approach: generate thermal resistances of key insulation components and run basic simulation

#Defining key variables for cooler design. This model IGNORES plastic conduction.
R_ice = 0.08 #guess for 500ml bottle
#first insulation layer between ice and vaccines
t_insulation = 0.005 #stops vaccine freezing
R_insulation = R_ice + t_insulation 
k_insulation = 0.03
Rth_insulation = Rth_radial(R_ice, R_insulation, k_insulation)
#air gap containing vaccines
t_air = 0.04
R_air = R_insulation + t_air
k_air = 0.024
Rth_air = Rth_radial(R_insulation, R_air, k_air)
#foam outer layer 
t_foam = 0.08
R_foam = R_air + t_foam
k_foam = 0.03
Rth_foam = Rth_radial(R_air, R_foam, k_foam)
#printing total part diameter, for refernce:
print(str(R_foam*2*100) + "cm")



41.0cm


In [99]:
#Convection loss estimation
v = 1 #this is speed of external air
h_outer = 12.12-1.16*v+11.6*v**(1/2) # #empirical relationship https://www.engineeringtoolbox.com/convective-heat-transfer-d_430.html - this is an empirical equation and can be used for velocities 2 to 20 m/s.
Rth_convection = 1/(2*np.pi*R_foam*l*h_outer)

#total thermal resistance 
Rth_total =  Rth_foam + Rth_air + Rth_insulation + Rth_convection

print(Rth_convection)

print(Rth_total)


0.17206683866534264
27.68992903620713


In [100]:
#Flat surface simulation
#Constructed of insulated and foam layer
def Rth_linear(l, A, k):
    Rth = l/(A*k)
    return Rth

A = np.pi * R_ice * R_ice

#first insulation layer, the same layer used around ice bottle circumfrence
t_flat_insulation = t_insulation

Rth_Flat_Insulation = Rth_linear(t_flat_insulation, A, k_insulation)

#second insulation layer 
t_flat_foam = 0.05

Rth_flat_foam = Rth_linear(t_flat_foam, A , k_insulation)

Rth_flat_total = Rth_Flat_Insulation + Rth_flat_foam



In [101]:
#setting up simulation
T_e = 30 #degc, environment temp
T_ice = 0 #degc, ice temp
del_T = T_e - T_ice
Q_dot_radial = del_T / Rth_total #using ohms analogy
Q_dot_linear = 2 * del_T / Rth_flat_total
Q_dot = Q_dot_linear + Q_dot_radial
print(Q_dot)

1.7414472564955976


In [102]:
#Estimating phase change duration 
m_ice = 2.5 #kg
SLH_ice = 334000 #j/kg
Q_ice = SLH_ice * m_ice

t_cool = Q_ice/Q_dot # seconds
t_cool_hours = t_cool / 3600

print(t_cool_hours)

133.19062267277505


In [103]:
#Calculating temperature at each boundary 
del_T_insulation = Q_dot * Rth_insulation 
print(del_T_insulation)
del_T_air = Q_dot * Rth_air
T_air_edge = del_T_air + del_T_insulation
print(T_air_edge)

2.8004527474749965
25.06924225964227
